In [ ]:
import xml.dom.minidom as minidom
import xml.etree.ElementTree as xmlET

### Create a code block to repeat
This block of code is boilerplate code that needs to be repeated for each parameter,

In [ ]:
code_block = """call this%output_variables%set('{0}', '{1}', {2}, '{3}', '{4}')"""
# code_block = """case('{0}')
#     this%{0}%name = last
#     call this%{0}%read(iUnit)
#     line = line + this%{0}%size() + this%{0}%dim_names%size() + ENTRY_OFFSET"""

### Get list of sorted variables 

In [ ]:
workfile = '<path>/prms6/src/xml/variables.xml'

xml_tree = xmlET.parse(workfile)
xml_root = xml_tree.getroot()

print(xml_root.tag)

In [ ]:
var_dict = {}

# Iterate over child nodes of root
for elem in xml_root.findall('variable'):
    # print(elem.attrib.get('name'))
    name = elem.attrib.get('name')
    dtype = elem.find('type').text
    desc = elem.find('desc').text
    units = elem.find('units').text
    
    # Add dimensions for current parameter
    dims_list = []
    for cdim in elem.findall('./dimensions/dimension'):
        dims_list.append(cdim.attrib.get('name'))
    
    dims = ','.join(dims_list)
    # Print the elements of the node
    # print(elem.find('desc').text)
    # print(elem.find('size').text)
    # dim_size = int(elem.find('size').text)
    var_dict[name] = [dtype, desc, units, dims]
    
variables = var_dict.keys()
variables.sort()


In [ ]:
# Open files to write the typedefs and code blocks to
basedir = '<path>/prms6'
code_hdl = open('{}/src/prmslib/classes/sm_output_variables_code.inc'.format(basedir), 'w')

tb = '  '

### Generate the repeated code

In [ ]:
datatype_map = {'I': 4, 'F': 5, 'D': 6, 'S': 2}
top_comment = '! Datatypes: NC_FLOAT=5, NC_DOUBLE=6, NC_INT=4, NC_CHAR=2'
code_hdl.write('{}\n'.format(top_comment))

for vv in variables:
    code_hdl.write('{}\n'.format(code_block.format(vv, var_dict[vv][3], datatype_map[var_dict[vv][0]], var_dict[vv][1], var_dict[vv][2])))
#     print(code_block.format(pp))
    
# code_hdl.write(code_footer)
code_hdl.write('\n')
code_hdl.close()

### Create code blocks for defining the parameter datatypes
This uses a parameter to datatype mapping which was generated by Generate_PRMS_valid_parameters_by_module-Copy1.

In [ ]:
datatype_map = {'I': 'iVariable', 'F': 'rVariable', 'S':'sVariable'}

### Define the code block for parameter type definitions

In [ ]:
# type(rVariable) :: K_coef
def_block = """type({}) :: {}"""

### Generate the type definition code blocks

In [ ]:
for pp in parameters:
    typedef_hdl.write('{}\n'.format(def_block.format(datatype_map[param_dict[pp]], pp)))
#     print(def_block.format(datatype_map[param_dict[pp]], pp))
typedef_hdl.write('\n')
typedef_hdl.close()

In [ ]:
import netCDF4

In [ ]:
NETCDF_DATATYPES